# Get them adopted

![Austin-shelter](http://www.austintexas.gov/sites/default/files/aac_logo.jpg)

## Business question:

Austin, Texas is the largest No Kill community in the nation, and home to the Austin Animal Center. They provide shelter to more than 16,000 animals each year and animal protection and pet resource services to all of Austin and Travis County. Annually over 90% of animals entering the center, are adopted, transferred to rescue or returned to their owners.

The Austin Animal Center _also_ has the longest running public dataset of animal rescues in the country. Every animal has a unique id that ties to both their [intake](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm) and [outcome](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Outcomes/9t4d-g238) data when they enter and leave the shelter. Given their data, my questions of interest were:

- What animals need a PR campaign?
- What resources does the Animal Center need?

Analyzing the Austin Animal Center data to answer these questions will assist their mission an improve operations. 

### Defining "Needing PR campaign" as animals that take the longest time to get adopted, or have the fewest transfers to rescue partners.
![img](./images/animals.png)


### For resources, I chose to focus on factors that would impact **cost** and _staffing_:
- breed/specialization of staff
- funds for neutering animals
- average stay of animals - would impact resources needed (food, cleaning, space)
- condition of animals when they enter the shelter
- percent of "special needs" animals - babies/seniors

![img2](./images/pet-resource-center-og.jpg)

### Data

### Methods

### Import packages and process data

`data_cleaning` and `feature_engineering` are packages made to store the functions and code used to prepare the data in seperate namespaces. 

In [112]:
import pandas as pd
import numpy as np
import code_folder.data_cleaning as dc
import code_folder.feature_engineering as fe
import code_folder.visualizations_graphs as viz_g
import code_folder.visualizations_tables as viz_t

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

# %load_ext autoreload
# %autoreload 2


In [3]:
outcomes = pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
intakes = pd.read_csv('https://data.austintexas.gov/api/views/wter-evkm/rows.csv?accessType=DOWNLOAD')

#### `data_cleaning`
For each intake and outcome file, respectively addresses:
- dates not in the correct datetime format
- column names have mixed cases and spaces 
- an age variable that needs to be replaced
- unnecessary columns tht can be dropped
- create some additional date variables to help merge later

Before merging the datasets to create a record for each animal's stay each year. Length of stay can then be created by comparing the intake and outcome dates.

In [4]:
outcomes = dc.prep_outcomes_file(outcomes)
intakes = dc.prep_intakes_file(intakes)
animal_shelter_df = dc.merge_files(intakes,outcomes)

#### `feature_engineering`

In [5]:
animal_shelter_df = fe.create_species_var(animal_shelter_df)
animal_shelter_df = fe.create_neutered_var(animal_shelter_df)
animal_shelter_df = fe.reduce_breed_list(animal_shelter_df)

In [ ]:
animal_shelter_df.to_csv('./data/cleaned_animal_center_df.csv')

##### `create_species_var`

Exploritory analysis revealed that the two variables that describe the "type" of animal were `animal_type` and `breed`.

While Cat and Dog were straight forward, "Other" appeared to hold a mix of species - both pets and wildlife. The snippet below also shows there is a lot of fuzzy matching within the breed variable. "Rabbit" and "Guinea Pig" each had repeat entries. I created `create_species_var` to map and aggregate the breed and intake_type to provide more insight into the types of animals the shelter services.

##### `create_neutered_var` and `reduce_breed_list`

 Neutering of animals at the Austin Animal Center is funded by [The City of Austin and Travis County](https://www.austintexas.gov/page/no-kill-plan), but how many neuterings are they doing a month? Is that number increasing  or decreasing? By comparing the sex of the animal at intake versus the outcome, a change in status from "intact" to "neutered" reveals the animal was neutered during their stay at the Rescue Center.
 
The reduced_breed_list removed "mix" from many of the cat and dog breeds. This allowed "Domestic Shorthair Mix" cats to be counted along with "Domestic Shorthair" and the same with "German Shepard Mix" being count as a "German Shepard."

#### Revisit our questions

- Age of animals in shelter
- Average animal length of stay
- Medical staff needed

Going to exclude livestock - only 12 and they are pigs and goats sample story animal_id = 103883, 6153 - they get transfered or partner org

ducks: 72688, 51609

In [33]:
## Creates table visualizations
viz_t.value_counts_table(animal_shelter_df.species.value_counts().append(pd.Series(animal_shelter_df.shape[0], index=["Total"])), "Animal Intake since 2013", "species_i")
viz_t.value_counts_table(animal_shelter_df.loc[animal_shelter_df.year==2019].species.value_counts().append(pd.Series(animal_shelter_df.loc[animal_shelter_df.year==2019].shape[0], index=["Total"])), "Animal Intake since 2013", "intake_2019")

In [34]:
## Creates time series graohs of intakes by animal species/type

# Get intake count of cats and dogs over time
summed_df = animal_shelter_df.loc[(animal_shelter_df['species'].str.contains("Dog|Cat"))].groupby([pd.Grouper(key="date_i",freq='1M'), 'species']).count()\
                .reset_index().pivot(index='date_i', columns='species', values='animal_id')
viz_g.time_series_plot(summed_df,"Seasonal Intake patterns of Cats and Dogs","Date by Month","Intake Count","intake_cat_dog",100)

# Get intake count of wildlife over time
summed_wildlife = animal_shelter_df.loc[(animal_shelter_df['species'].str.contains("Wildlife"))].groupby([pd.Grouper(key="date_i",freq='1M'), 'species']).count()\
                .reset_index().pivot(index='date_i', columns='species', values='animal_id')
viz_g.time_series_plot(summed_wildlife,"Intake patterns of Wildlife","Date by Month","Intake Count",'wildlife_count',10)

# Get intake count of all others over time
summed_df_small_b = animal_shelter_df.loc[(~animal_shelter_df['species'].str.contains("Dog|Cat|Wildlife"))].groupby([pd.Grouper(key="date_i",freq='1M'), 'species']).count()\
                .reset_index().pivot(index='date_i', columns='species', values='animal_id')
viz_g.time_series_plot(summed_df_small_b,"Intake patterns of Animals that aren't Cats or Dogs","Date by Month","Intake Count",'other_species_count',10)

# Highlight history of Chickens, Rabbits, and Guinea Pigs over time
summed_df_small_pets = animal_shelter_df.loc[(animal_shelter_df['species'].str.contains("Chicken|Rabbit|Guinea"))].groupby([pd.Grouper(key="date_i",freq='1M'), 'species']).count()\
                .reset_index().pivot(index='date_i', columns='species', values='animal_id')
viz_g.time_series_plot(summed_df_small_pets,"Intake History of Chickens, Rabbits, and Guinea Pigs","Date by Month","Intake Count",'small_pet_count',5)

<Figure size 1280x800 with 0 Axes>

<Figure size 1280x800 with 0 Axes>

<Figure size 1280x800 with 0 Axes>

<Figure size 1280x800 with 0 Axes>

In [60]:
test = animal_shelter_df.loc[(animal_shelter_df['species'].str.contains("Dog|Cat")) & (animal_shelter_df.neutered ==1)].groupby([pd.Grouper(key="date_i",freq='1M'), 'species']).count()\
                .reset_index().pivot(index='date_i', columns='species', values='animal_id')

test.columns = ['Cat_neutered', "Dog_neutered"]

test2 = pd.concat([summed_df, test], axis=1)

test2['percent_cat'] = test2.Cat_neutered/test2.Cat
test2['percent_dog'] = test2.Dog_neutered/test2.Dog
test2.drop(columns = ['Cat', 'Dog', 'Cat_neutered', 'Dog_neutered'], inplace = True)
test2.columns = ['Cat', 'Dog']


In [77]:
viz_g.time_series_plot_percent(test2,"History of Percent Neutered at Intake" ,"Date by Month","Percent Neutered","neutered_over_time2", 0.05)

<Figure size 1280x800 with 0 Axes>

## Animal Intake Overall

| ![all3](./images/species_i.png) | ![2019a](./images/intake_2019.png)

## Animal Intake Trends

![one](./images/intake_cat_dog.png) 
![two](./images/wildlife_count.png)
![three](./images/other_species_count.png)


### Only non-seasonal pattern
![foura](./images/small_pet_count.png)

## Neutering patterns

![neut](./images/neutered_over_time2.png)

## Medical Staffing needs:

### Findings: 

### Intake Patterns:
- The intake of cats and dogs seems to have a seasonal pattern that remains relatively constant
- Wildlife also seems fairly seasonal, with perhaps a slight downward trend since 2017
- For those animals who have seasonal patterns, winter is a low time for all
- Rabbits, chickens, and guinea pigs do not appear to have a seasonal patter
- Rabbits, chickens, and guinea pigs *do* appear to have sudden spikes

### Neutering:
- about 30-55% of cats who come in are neutered during their stay
- about 40-55% of dogs who come in are neutered during their stay, smaller variance than cats

### Comparison of Outcomes for Neutered vs Intact

In [107]:
intact_count = animal_shelter_df.loc[(animal_shelter_df.species.str.contains("Dog|Cat")) & (animal_shelter_df.sex_upon_outcome.str.contains("Intact"))].outcome_type.value_counts(normalize=True).sort_index()
intact_count.name = "percent_intact"

In [108]:
neutered_count = animal_shelter_df.loc[(animal_shelter_df.species.str.contains("Dog|Cat"))&(animal_shelter_df.sex_upon_outcome.str.contains("Neutered|Spayed"))].outcome_type.value_counts(normalize=True).sort_index()
neutered_count.name = "percent_neutered"

In [109]:
compare_neuter_outcome = pd.DataFrame(data={"intact":intact_count,"neutered":neutered_count})

In [110]:
compare_neuter_outcome

,intact,neutered
Adoption,0.081870,0.643787
Died,0.023333,0.001901
Disposal,0.001274,0.000119
Euthanasia,0.046893,0.012500
Missing,0.001637,0.000277
Return to Owner,0.211953,0.167573
Rto-Adopt,0.002456,0.008316
Transfer,0.630583,0.165527


## Medical needs

1. How many animals come in injured? And what happens to them?
2. How many animals come in and over their stay get neutered?
3. Any trends in the animals that do not get neutered?

### Recommendations:



### Business Recommendations

### Limitations

- Missing animals that came back to the shelter more than one time during a year
- Not examining animals still in the shelter
- Do not know specific costs of care or current staffing of Rescue Center
- Consistent dips in January and December might be due to how we merged the data on year, so artifical dip


### Next Steps

